<a href="https://colab.research.google.com/github/Chichi213/Daiz_CPE-313/blob/main/Daiz_Hands_on_Activity_2_2_Transfer_Learning_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Activity 2.2 - Transfer Learning

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [17]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [18]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [19]:

if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [20]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [21]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()



Create two datasets
* one with digits below 5
* one with 5 and above

In [22]:
# digit 0 to 4
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]
# digit 5-9
x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [23]:
# Feature layer
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [24]:
#Classification layer
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [25]:
model = Sequential(feature_layers + classification_layers)

Check the model summary

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

 Train the  model on the digits 5,6,7,8,9

In [27]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 44s 188ms/step - loss: 1.6064 - accuracy: 0.2091 - val_loss: 1.5842 - val_accuracy: 0.2674
Epoch 2/5
230/230 [==============================] - 47s 205ms/step - loss: 1.5781 - accuracy: 0.2678 - val_loss: 1.5525 - val_accuracy: 0.4131
Epoch 3/5
230/230 [==============================] - 45s 194ms/step - loss: 1.5478 - accuracy: 0.3348 - val_loss: 1.5174 - val_accuracy: 0.6098
Epoch 4/5
230/230 [==============================] - 45s 194ms/step - loss: 1.5143 - accuracy: 0.4026 - val_loss: 1.4779 - val_accuracy: 0.7159
Epoch 5/5
230/230 [==============================] - 45s 194ms/step - loss: 1.4760 - accuracy: 0.4727 - val_loss: 1.4332 - val_accuracy: 0.7558
Training time: 0:03:45.566301
Test score: 1.433160662651062
Test accuracy: 0.7558115720748901


Freeze only the feature layers

In [28]:
for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [29]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

Train again the model using the 0 to 4 digits

In [30]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 18s 74ms/step - loss: 1.5803 - accuracy: 0.3024 - val_loss: 1.5354 - val_accuracy: 0.5260
Epoch 2/5
240/240 [==============================] - 16s 68ms/step - loss: 1.5238 - accuracy: 0.3974 - val_loss: 1.4792 - val_accuracy: 0.6285
Epoch 3/5
240/240 [==============================] - 16s 68ms/step - loss: 1.4742 - accuracy: 0.4820 - val_loss: 1.4264 - val_accuracy: 0.6852
Epoch 4/5
240/240 [==============================] - 15s 63ms/step - loss: 1.4271 - accuracy: 0.5512 - val_loss: 1.3752 - val_accuracy: 0.7387
Epoch 5/5
240/240 [==============================] - 15s 64ms/step - loss: 1.3791 - accuracy: 0.6152 - val_loss: 1.3251 - val_accuracy: 0.7918
Training time: 0:01:22.780245
Test score: 1.325149655342102
Test accuracy: 0.7917882800102234


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [33]:
for l in feature_layers:
    l.trainable = False

In [34]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 18s 71ms/step - loss: 1.4321 - accuracy: 0.4769 - val_loss: 1.3862 - val_accuracy: 0.6248
Epoch 2/5
230/230 [==============================] - 17s 74ms/step - loss: 1.3942 - accuracy: 0.5323 - val_loss: 1.3460 - val_accuracy: 0.7003
Epoch 3/5
230/230 [==============================] - 15s 65ms/step - loss: 1.3596 - accuracy: 0.5751 - val_loss: 1.3056 - val_accuracy: 0.7535
Epoch 4/5
230/230 [==============================] - 19s 83ms/step - loss: 1.3203 - accuracy: 0.6229 - val_loss: 1.2654 - val_accuracy: 0.7776
Epoch 5/5
230/230 [==============================] - 18s 80ms/step - loss: 1.2834 - accuracy: 0.6551 - val_loss: 1.2259 - val_accuracy: 0.7988
Training time: 0:02:23.604491
Test score: 1.225924015045166
Test accuracy: 0.7988068461418152


#### Conclusion

"For this activity, we are tasked with performing transfer learning, as the name suggests, it saves the last trained model. As I observed during training throughout this activity, in our first training data, the accuracy were rather low, indicating that the model didn't learn our data well. However, during the second training session, the model improved, achieving an accuracy of 79%. In the third training session, the accuracy of the model increased to 80% when rounded to the nearest decimal point. Overall, this activity helped me better understand how transfer learning works by increasing accuracy when the model is trained repeatedly. This method will assist me in future activities and research.